In [ ]:
import os
from owslib.wms import WebMapService
import json

def save_all_metadata(metadata_list, filename):
    with open(filename, "w") as f:
        json.dump(metadata_list, f, indent=4)

def download_image(wms, layer, bbox, size, format, filename, crs, resolution, metadata_list):
    response = wms.getmap(
        layers=[layer],
        srs=crs,
        bbox=bbox,
        size=size,
        format=format,
        transparent=True
    )
    # Bild speichern
    img = Image.open(BytesIO(response.read()))
    img.save(filename)
    
    # Metadaten für das Bild
    metadata = {
        "filename": filename,
        "bbox": bbox,
        "size": size,
        "crs": crs,
        "resolution": resolution
    }
    
    # Metadaten zur Liste hinzufügen
    metadata_list.append(metadata)

def collect_metadata(wms, layer, bbox, size, format, crs, resolution, metadata_list):
    # Metadaten sammeln, ohne das Bild herunterzuladen
    metadata = {
        "filename": f"tile_{int(bbox[0])}_{int(bbox[1])}",
        "bbox": bbox,
        "size": size,
        "crs": crs,
        "resolution": resolution
    }
    
    # Metadaten zur Liste hinzufügen
    metadata_list.append(metadata)

# WMS-Server-URL
url = "https://geodienste.bremen.de/wms_dop10_2023"

# Verbindung zum WMS-Server herstellen
wms = WebMapService(url, version='1.3.0')

# Layer auswählen
layer = 'DOP10_2023_HB'

# Bounding Box definieren (minx, miny, maxx, maxy)
bbox = (488608.7545, 5881989.4935, 490215.9319, 5883625.7616)

# Zielgröße der Kacheln in Pixeln
tile_size = 320

# Zielauflösung in Metern pro Pixel
resolution = 0.1  # Beispielwert, bitte anpassen

# CRS
crs = 'EPSG:25832'

# Breite und Höhe der Bounding Box in Metern
bbox_width = bbox[2] - bbox[0]
bbox_height = bbox[3] - bbox[1]

# Anzahl der Kacheln in x- und y-Richtung
num_tiles_x = int(bbox_width / (tile_size * resolution))
num_tiles_y = int(bbox_height / (tile_size * resolution))

# Breite und Höhe jeder Kachel in Metern
tile_width = bbox_width / num_tiles_x
tile_height = bbox_height / num_tiles_y

# Liste für alle Metadaten
metadata_list = []

# Iteration über alle Tiles, um nur die Metadaten zu sammeln
for i in range(num_tiles_y):
    for j in range(num_tiles_x):
        tile_bbox = (
            bbox[0] + j * tile_width,
            bbox[1] + i * tile_height,
            bbox[0] + (j + 1) * tile_width,
            bbox[1] + (i + 1) * tile_height
        )
        
        # Metadaten sammeln
        collect_metadata(wms, layer, tile_bbox, (tile_size, tile_size), 'image/png', crs, resolution, metadata_list)
        
        # Bilder herunterladen
        download_image(wms, layer, tile_bbox, (tile_size, tile_size), 'image/png', f"tile_{int(tile_bbox[0])}_{int(tile_bbox[1])}.png", crs, resolution, metadata_list)

# Alle Metadaten in einer JSON-Datei speichern
save_all_metadata(metadata_list, "metadata.json")

print("Bilder heruntergeladen und Metadaten erfolgreich gesammelt und in einer JSON-Datei gespeichert.")